## Setup for Music21

In [1]:
# Check which version of Linux we have, and our distribution.
!uname
!uname -r
!lsb_release -a

Linux
4.14.65+
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.1 LTS
Release:	18.04
Codename:	bionic


In [2]:
# Updates the Linux package installer thing we need to use.
!apt upgrade

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn7 libnccl2
The following packages will be upgraded:
  dpkg libsystemd0 libudev1
3 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Need to get 1,398 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 dpkg amd64 1.19.0.5ubuntu2.1 [1,140 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libsystemd0 amd64 237-3ubuntu10.4 [204 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libudev1 amd64 237-3ubuntu10.4 [53.8 kB]
Fetched 1,398 kB in 3s (536 kB/s)
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../dpkg_1.19.0.5ubuntu2.1_amd64.deb ...
Unpacking dpkg (1.19.0.5ubuntu2.1) over (1.19.0.5ubuntu2) ...
Setting up dpkg (1.19.0.5ubuntu2

In [3]:
# Allows us to install directly from source, as we're about to do for Lilypond.
!sed -i '/deb-src/s/^# //' /etc/apt/sources.list && apt update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Get:3 http://archive.canonical.com/ubuntu bionic InRelease [10.2 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release [564 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release.gpg [801 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://archive.canonical.com/ubuntu bionic/partner Sources [1,906 B]
Get:13 https://developer.download.nvidia.com/compute/machine-l

In [4]:
# Build the dependencies for Lilypond.
!apt build-dep lilypond

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  autoconf automake autopoint autotools-dev bison bsdmainutils dblatex
  debhelper dh-autoreconf dh-exec dh-strip-nondeterminism diffstat docbook-xml
  file flex fontconfig fontforge fontforge-common fonts-freefont-otf
  fonts-gfs-baskerville fonts-gfs-porson fonts-ipafont-mincho
  fonts-linuxlibertine fonts-lmodern gettext gettext-base ghostscript
  gir1.2-freedesktop gir1.2-pango-1.0 groff groff-base gsfonts help2man
  hicolor-icon-theme imagemagick imagemagick-6-common imagemagick-6.q16
  intltool-debian libarchive-zip-perl libavahi-client3 libavahi-common-data
  libavahi-common3 libbison-dev libcairo-gobject2 libcairo-script-interpreter2
  libcairo2 libcairo2-dev libcroco3 libcups2 libcupsimage2 libdatrie1
  libfftw3-double3 libfile-stripnondeterminism-perl libfl-dev libfl2
  libfontforge2 libgdraw5 libgif7 libgmp-dev libgmpxx4ldbl libgs9
  l

In [5]:
# Lilypond also wants us to run this.
!apt install autoconf fonts-texgyre texlive-lang-cyrillic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
autoconf is already the newest version (2.69-11).
texlive-lang-cyrillic is already the newest version (2017.20180305-1).
The following NEW packages will be installed:
  fonts-texgyre
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 8,761 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-texgyre all 20160520-1 [8,761 kB]
Fetched 8,761 kB in 3s (2,836 kB/s)
Selecting previously unselected package fonts-texgyre.
(Reading database ... 62313 files and directories currently installed.)
Preparing to unpack .../fonts-texgyre_20160520-1_all.deb ...
Unpacking fonts-texgyre (20160520-1) ...
Processing triggers for tex-common (6.09) ...
Running mktexlsr. This may take some time... done.
Setting up fonts-texgyre (20160520-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu

In [6]:
# Now install Lilypond.
!apt install lilypond

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  lilypond-data
Suggested packages:
  lilypond-doc
The following NEW packages will be installed:
  lilypond lilypond-data
0 upgraded, 2 newly installed, 0 to remove and 2 not upgraded.
Need to get 3,727 kB of archives.
After this operation, 16.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 lilypond-data all 2.18.2-12build1 [1,799 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 lilypond amd64 2.18.2-12build1 [1,928 kB]
Fetched 3,727 kB in 3s (1,294 kB/s)
Selecting previously unselected package lilypond-data.
(Reading database ... 62533 files and directories currently installed.)
Preparing to unpack .../lilypond-data_2.18.2-12build1_all.deb ...
Unpacking lilypond-data (2.18.2-12build1) ...
Selecting previously unselected package lilypond.
Preparing to unpack .../lilyp

In [7]:
# Also install Musescore.
!apt install musescore

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core avahi-daemon bind9-host dbus
  dconf-gsettings-backend dconf-service desktop-file-utils dmsetup
  fluidr3mono-gm-soundfont geoclue-2.0 geoip-database glib-networking
  glib-networking-common glib-networking-services gsettings-desktop-schemas
  gstreamer1.0-plugins-base gtk-update-icon-cache humanity-icon-theme
  iio-sensor-proxy libapparmor1 libargon2-0 libasound2 libasound2-data
  libasyncns0 libatk-bridge2.0-0 libatk1.0-0 libatk1.0-data libatspi2.0-0
  libavahi-core7 libavahi-glib1 libbind9-160 libbrotli1 libcap2 libcap2-bin
  libcdparanoia0 libcolord2 libcryptsetup12 libdaemon0 libdconf1
  libdevmapper1.02.1 libdns1100 libdouble-conversion1 libdrm-amdgpu1
  libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libepoxy0 libevdev2 libflac8
  libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-bin libgdk-pixbuf2.0-common
  libgeo

In [8]:
!whereis musescore
!whereis lilypond

musescore: /usr/bin/musescore
lilypond: /usr/bin/lilypond.real /usr/bin/lilypond /usr/lib/x86_64-linux-gnu/lilypond /usr/share/lilypond


# Import Libraries and Data

In [9]:
!pip install music21

    100% |████████████████████████████████| 18.5MB 2.1MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/7b/21/95/d396f231b8095f30aba2a1fbffbc2411fb22eb4e611ddbed57
Successfully built music21


In [0]:
from music21 import converter, environment, harmony
import glob

In [0]:
environment.UserSettings()['lilypondPath'] = '/usr/bin/lilypond'
environment.set('pdfPath', '/usr/bin/musescore')
environment.set('graphicsPath', '/usr/bin/musescore')

In [12]:
!pip install -q xlrd
!git clone https://github.com/Benendead/LSTMjazz/

Cloning into 'LSTMjazz'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 82 (delta 6), reused 13 (delta 2), pack-reused 62
Unpacking objects: 100% (82/82), done.


In [13]:
# Check out the directory.
!ls LSTMjazz

Datasets  Keyboard.py  README.md


In [14]:
# Find the songs.
!ls LSTMjazz/Datasets/"Omnibook Mxml"

 An_Oscar_For_Treadwell.xml   Kim_2.xml
 Another_Hairdo.xml	      Ko_Ko.xml
 Anthropology.xml	      Laird_Baird.xml
 Au_Private_1.xml	      Marmaduke.xml
 Au_Private_2.xml	      Mohawk_1.xml
 Back_Home_Blues.xml	      Mohawk_2.xml
 Barbados.xml		      Moose_The_Mooche.xml
"Billies's_Bounce.xml"	      My_Little_Suede_Shoes.xml
 Bird_Gets_The_Worm.xml      "Now's_The_Time_1.xml"
 Bloomdido.xml		     "Now's_The_Time_2.xml"
 Blue_Bird.xml		      Ornithology.xml
 Blues_For_Alice.xml	      Passport.xml
 Buzzy.xml		      Perhaps.xml
 Card_Board.xml		      Red_Cross.xml
 Celerity.xml		      Relaxing_With_Lee.xml
 Chasing_The_Bird.xml	      Scrapple_From_The_Apple.xml
 Cheryl.xml		      Segment.xml
 Chi_Chi.xml		      Shawnuff.xml
 Confirmation.xml	      Si_Si.xml
 Cosmic_Rays.xml	      Steeplechase.xml
 Dewey_Square.xml	      The_Bird.xml
 Diverse.xml		      Thriving_From_A_Riff.xml
 Donna_Lee.xml		      Visa.xml
 KC_Blues.xml		      Warming_Up_A_Riff.xml
 Kim_1.xml		      Yardbird_Suite.xml


In [15]:
# Import the songs into a list.
test = True # Set to true to only import one song.

untransposedSongs = []
i = 1
for xml_file in glob.glob("LSTMjazz/Datasets/*/*.xml"): # Selects all .xml files in the Datasets folder.
    print('Processing song ', i, ":", xml_file)
    uTsong = converter.parse(xml_file)
    untransposedSongs.append(uTsong)
    if (test and (i == 1)):
      break
    i = i + 1

Processing song  1 : LSTMjazz/Datasets/Omnibook Mxml/My_Little_Suede_Shoes.xml


In [39]:
!pip install Wand

    100% |████████████████████████████████| 71kB 1.4MB/s 
  Running setup.py bdist_wheel for Wand ... - done
  Stored in directory: /root/.cache/pip/wheels/69/27/bf/e3a7915d1ac83745a9486cb50f1cb0d676cc8642ae711193a6
Successfully built Wand


In [44]:
from wand.image import Image as WImage
img = WImage(stream0.write("lily.png"))
img

TypeError: ignored

In [45]:
file = stream0.write("lily.png")
type(file)

pathlib.PosixPath

In [46]:
type(file)

pathlib.PosixPath

In [0]:
stream0 = untransposedSongs[0]
harmony.realizeChordSymbolDurations(stream0)
stream0.show("lily.png")

In [38]:
from IPython.display import Image
Image(filename = stream0.write("lily.png"))

AttributeError: ignored

In [0]:
# Transposes the songs into C.

counter = 1
songs = []
for s in untransposedSongs:
    #print ("Transposing song", counter)
    key = s.analyze('key')
    i = interval.Interval(key.tonic, pitch.Pitch('C')) # Finds how far off we are from C.
    sNew = s.transpose(i) # Transposes us that far to C.
    counter = counter + 1
    songs.append(sNew)

In [0]:
noteDuration = 0
chordDuration = 0
for song in songs:
  harmony.realizeChordSymbolDurations(song)
  toParse = song.flat.recurse() # recurse() goes over all objects in the Music21 stream.
      
  for i in range(len(toParse)):
    if isinstance(toParse[i], note.Note):
      noteDuration += float(toParse[i].quarterLength)
      if(toParse[i-1].quarterLength < abs(toParse[i].offset - toParse[i-1].offset)):
        noteDuration += abs (-toParse[i-1].offset + toParse[i].offset)
    elif isinstance(toParse[i], chord.Chord):
      chordDuration += float(toParse[i].quarterLength)
  
  print("Song:",song.highestTime)
  print("Duration sum:", noteDuration)
  
  noteDuration = 0;
  chordDuration = 0;

Song: 392.0
Duration sum: 337.00000000000006
Song: 248.0
Duration sum: 204.33333333333366
Song: 264.0
Duration sum: 233.3333333333334
Song: 148.0
Duration sum: 117.28333333333349
Song: 264.0
Duration sum: 208.50000000000003
Song: 260.0
Duration sum: 210.91666666666669
Song: 260.0
Duration sum: 202.33333333333346
Song: 264.0
Duration sum: 201.50000000000003
Song: 388.0
Duration sum: 330.6666666666673
Song: 152.0
Duration sum: 126.41666666666674
Song: 196.0
Duration sum: 159.66666666666669
Song: 196.0
Duration sum: 146.08333333333331
Song: 196.0
Duration sum: 167.00000000000017
Song: 200.0
Duration sum: 140.8166666666668
Song: 196.0
Duration sum: 165.48333333333323
Song: 292.0
Duration sum: 241.58333333333343
Song: 524.0
Duration sum: 436.8833333333332
Song: 260.0
Duration sum: 197.5000000000002
Song: 260.0
Duration sum: 214.6666666666668
Song: 264.0
Duration sum: 194.16666666666688
Song: 164.0
Duration sum: 121.41666666666681
Song: 196.0
Duration sum: 165.99999999999997
Song: 388.0
Dura

In [0]:
# Converts fraction in a String to a float e.g. "1/2" to 0.5

def fracToFloat(fracString):
  try:
    return float(fracString)
  except ValueError:
    num, denom = fracString.split('/')
    try:
      leading, num = num.split(' ')
      whole = float(leading)
    except ValueError:
      whole = 0
      frac = float(num) / float(denom)
      return whole - frac if whole < 0 else whole + frac

In [0]:
def emptyArray(size):
  ar = [0]
  for x in range(size - 1):
    ar.append(0)
  return ar

In [0]:
# Parsing the songs for notes, offsets, and durations.

notes = []
noteOffsets = []
chords = []
temp = []
chordOffsets = []
noteDuration = []
chordDuration = []


for song in songs:
  harmony.realizeChordSymbolDurations(song)
  notes_to_parse = song.recurse() # recurse() goes over all objects in the Music21 stream.
      
  for element in notes_to_parse:
    tempChordPitches = []
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
      noteOffsets.append(fracToFloat(element.offset))
      noteDuration.append(float(element.quarterLength))
    elif isinstance(element, chord.Chord):
      for n in element.normalOrder:
        tempChordPitches.append(n)
      chords.append(tempChordPitches)
      chordOffsets.append(str(element.offset))
      chordDuration.append(float(element.quarterLength))

noteLength = 0
chordLength = 0

for i in range(len(notes)):
  noteLength += noteDuration[i]
for i in range(len(chords)):
  chordLength += chordDuration[i]
print((noteLength - chordLength))  

-2947.0833333333285


In [0]:
# Tells us the total duration of all songs based on original imports and summed chord duration.

total = 0
for song in songs:
  total += song.duration.quarterLength
  
print(total)

overAllLengthChords = 0
for c in range(len(chords)):
  overAllLengthChords += chordDuration[c]

print(overAllLengthChords)

14084.0
14084.0


In [0]:
# Changes chords into 1-hot encoded arrays
chordsCheck = ["C", "D-", "D", "E-", "E", "F", "G-", "G", "A-", "A", "B-", "B"]
oneChords = [[]]
oneChords[0] = emptyArray(12)
for i in range(len(chords)):
  oneChords.append(emptyArray(12))
  
index = 0
for chord in chords:
  for i in chord:
    oneChords[index][i] = 1
  index += 1

In [0]:
# Fixes enharmonic equivalents.

notes_to_change = ["C#", "D#", "E#", "F#", "G#", "A#", "F##"]
note_equivalents = ["D-", "E-", "F", "G-", "A-", "B-", "G"]

for note_number in range(len(notes)):
  #if note_number % 5000 == 0:
    #print("On note", note_number, "of", len(notes))
  if (notes[note_number] == "B#3"):
    notes[note_number] = "C4"
  if (notes[note_number] == "C-3"):
    notes[note_number] = "B2"
  if (notes[note_number] == "C-4"):
    notes[note_number] = "B3"
  if (notes[note_number] == "C-5"):
    notes[note_number] = "B4"
  if (notes[note_number] == "F-4"):
    notes[note_number] = "E4"
  if ("#" in notes[note_number]):
    for octave in range(2,6):
  
      for name_number in range(len(notes_to_change)):
        noteName = notes_to_change[name_number] + str(octave)
        if (noteName == notes[note_number]):
          new_name = note_equivalents[name_number] + str(octave)
          notes[note_number] = new_name

In [0]:
#Identify how many unique notes and chord durations there are
notesCheck = []
noteDurationCheck = []
chordsTotal= []
chordDurationCheck = []
for e in notes:
  if e not in notesCheck:
    notesCheck.append(e)

for e in noteDuration:
  if e not in noteDurationCheck:
    noteDurationCheck.append(e)
    
for e in chords:
  if e not in chordsTotal:
    chordsTotal.append(e)
    
for e in chordDuration:
  if e not in chordDurationCheck:
    chordDurationCheck.append(e)

print("Number of unique notes: ", len(notesCheck))
print("Number of unique note durations: ", len(noteDurationCheck))
print("Number of unique chords: ", len(chordsTotal))
print("Number of unique chord durations: ", len(chordDurationCheck))

for e in noteDurationCheck:
  print(e)

Number of unique notes:  41
Number of unique note durations:  15
Number of unique chords:  44
Number of unique chord durations:  4
0.5
1.0
0.3333333333333333
3.0
2.0
1.5
0.16666666666666666
0.25
0.75
0.125
0.2
4.0
0.6666666666666666
0.08333333333333333
0.8


In [0]:
# Finding the notes that are 4 beats.
totals = []
possible_lengths = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]

for length in possible_lengths:
  for i in range(len(noteDuration)):
    if noteDuration[i] == length:
      off = noteOffsets[i]
      total = off + length
      totals.append(total)
    
for totals in totals:
  if total != 4:
    print(total)

In [0]:
#Adds note to 1-hot encoded array
def addTo(ar, note):
  for i,j in enumerate(notesCheck):
    if(note == j):
      ar[i] = 1
      break

In [0]:
def getNote(ar):
  notes = []
  for i in range (len(notesCheck)):
    if ar[i] == 1:
      notes.append(str(notesCheck[i]))
  return notes

In [0]:
# Checks if a given chord is part of a new measure.
def newMeasure(c):
  if (c == 0):
    return False
  if (chordOffset[c-1] >= chordOffset[c]):
    return True
  return False

In [0]:
# Changes notes into 1-hot encoded arrays
oneNotes = [[]]
oneNotes[0] = emptyArray(42)
for i in range(len(notes)-1):
  oneNotes.append(emptyArray(42))
  
index = 0

for e in notes:
  for i,j in enumerate(notesCheck):
    if(e == j):
      oneNotes[index][i] = 1
      break
  if 1 not in oneNotes[index]:
    oneNotes[41] = 1
  index = index + 1

In [0]:
# Creates an array of Notes being played at each timestep
arrays = []
indexOfNotes = []
tempIndex = 0
noteIndex = 0
temp = []
index = 0
endMeasure = False
first = True

while noteIndex < len(notes):
  first = True
  for i in range(1,49): # Goes from 0 to 4 in 48 intervals.
    temp = []
    if(float(i/12) > noteOffsets[noteIndex] and not endMeasure):
      first = False
      for i in range(len(oneNotes[0])):
        temp.append(oneNotes[noteIndex][i])
        indexOfNotes.append(tempIndex)
      noteIndex += 1
    else:
      temp = emptyArray(42)
    arrays.append(temp)
    tempIndex+=1
    if(len(noteOffsets) - noteIndex <= 0):
      break
    if(noteOffsets[noteIndex] < noteOffsets[noteIndex-1] and not first):
      endMeasure = True
  
  endMeasure = False
  

print(len(arrays))
print(128945/48)

148945
2686.3541666666665


In [0]:
chordArrays = []
indexOfChords = []
tempIndex = 0
chordIndex = 0
test = [0,0,0]


while(chordIndex < len(chords)):
  for i in range(int(chordDuration[chordIndex]*12)):
    chordArrays.append(oneChords[chordIndex])
    if(i==0):
      indexOfChords.append(chordIndex)
  chordIndex+=1

print(chordDuration[0])  
for i in range(50):
  print("i: ", i, " " ,chordArrays[i])
print(len(chordArrays) - len(arrays))      

2.0
i:  0   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  1   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  2   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  3   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  4   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  5   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  6   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  7   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  8   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  9   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  10   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  11   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  12   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  13   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  14   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  15   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  16   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  17   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  18   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  19   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  20   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
i:  21   [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 

In [0]:
#Start of looping through timeSteps
counter = 0
length = float(1/12)
numerator = 0
index = 0
noteIndex = 0
# This doesn't work. I want to loop through the array of timeSteps, then if a timestep has a note, then adding that note to every timestep in its duration

#for i in range(0,10):
#  print("note: ", notes[i], " duration: ", noteDuration[i])    
noteIndex = 0

for i in range(len(arrays)):
  if (1 in arrays[i] and i == indexOfNotes[noteIndex]):
    numerator = 1
    length = float(numerator/12)
    #print("note Index: ", noteIndex, " noteDuration: ", noteDuration[noteIndex], " note: ", notes[noteIndex])
    for j in range(1,int(noteDuration[noteIndex]*12)):
      addTo(arrays[i+j],notes[noteIndex])
    noteIndex+=1
    if (noteIndex == len(notes)-1):
      break
    if(noteIndex % 100 == 0):
      print(noteIndex)
      
print("After")
noteIndex = 0
for i in range(50, 100):
  print(arrays[i])
  if 1 in arrays[i]:
    print(getNote(arrays[i]))
    noteIndex+=1

After
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['C4']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [0]:
#creates input layer

tracker = 0
input = []


for i in range(tracker,tracker+12):
  for j in chordArrays[tracker]:
    input.append(j)
    
    
print(len(input))      

144


In [0]:
overAllLengthNotes = 0
sumInMeasure = 0
measures = 1

for n in range(len(notes)):
  curr_offset = noteOffsets[n]
  curr_dur = noteDuration[n]
  if (curr_offset <= noteOffsets[n-1]): # New measure.
    overAllLengthNotes += sumInMeasure
    sumInMeasure = curr_dur
    measures += 1
  if (curr_offset > sumInMeasure): # Note after a rest.
    sumInMeasure = curr_dur + curr_offset
  else:
    sumInMeasure += curr_dur

print(overAllLengthNotes)

13720.05


In [0]:
# set measure counter to 0
# 
# go through all notes (while)
#     we have a note with duration and offset
#     use offset from the measure start to begin note
#     store note duration
#     calculate how many time steps the note is based on round dur / stepsize
#     set that many timesteps to the given note
#     if next note's offset is more
#          move to the next note's offset from the measure start
#     set that many steps to the note
#     
# method next note valid based on note number
# if not up measure

curr_offset = 0
measure = 0
timesteps = []
timestep = 0
for c in range(len(chord)):
  curr_step = []
  curr_offset = chordOffsets[c]
  curr_chord = chords[c]
  curr_dur = chordDuration[c]
  
  step = curr_chord
  